In [0]:
import torch
import torch.nn as nn
import numpy as np
import math
import torch.nn.functional as F
import argparse
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.optim as optim


class ChoiceNet(nn.Module):
  def __init__(self, backbone, y_dim, num_mixture, feature_dim, logSigmaZval, tau_inv, pi1_bias):
    super(ChoiceNet,self).__init__()
    self.backbone = backbone
    self.y_dim = y_dim
    self.num_mixture = num_mixture
    self.feature_dim = feature_dim
    self.logSigmaZval = logSigmaZval
    self.tau_inv = tau_inv
    self.pi1_bias = pi1_bias
    self.rho_ref = 1
    #self.USE_GAT = USE_GAT
    
    
    self.fc_feature_dim = nn.Linear(7*7*64, self.feature_dim)
    self.fc_num_mixture = nn.Linear(self.feature_dim, num_mixture)

    self.fc_var_raw = nn.Linear(self.feature_dim, self.y_dim)

    self.fc_pi_logits = nn.Linear(self.feature_dim, self.num_mixture)
  def forward(self, x):
    x = self.backbone(x)
    x = x.view(x.size(0),-1) # flatten
    self.feature = self.fc_feature_dim(x) # feature, h
    rho_raw = self.fc_num_mixture(self.feature)
    rho_temp = F.sigmoid(rho_raw)
    
    rho = torch.cat([rho_temp[:, 0:1]*0.0 + self.rho_ref, rho_temp[:, 1:]], axis=1) # rho(h)=rho1~rhoK, rho_ref=1

    Q = self.feature_dim
    num_data = x.size()[0]

    #make_sample
    muW_tile, muZ_tile, sigmaW_tile, sigmaZ_tile = self.make_sample(Q, num_data) 
    
    # cholesky #[K*N*Q*D]
    samplerList = self.cholesky(self.num_mixture, Q, rho, num_data, muW_tile, sigmaW_tile, muZ_tile, sigmaZ_tile)

    wSample = samplerList.permute(1,3,0,2) #[N*D*K*Q]

    #K mean mixtures
    wTemp = wSample.contiguous().view(num_data, self.num_mixture*self.y_dim, Q)
    #wTemp = torch.reshape(wSample,(wSample.shape[0],wSample.shape[1]*wSample.shape[2],wSample.shape[3])))
    #wTemp = wSample.view(num_data, self.num_mixture*self.y_dim, Q) #[N*DK*Q]
    featRsh = self.feature.view(num_data, Q, 1)
     
    _mu = torch.matmul(wTemp, featRsh) #[N*DK*1]
    mu = _mu.view(num_data, self.y_dim, self.num_mixture)

    ### Add bias to mu (after)

    #K var mixtures
    logvar_raw = self.fc_var_raw(self.feature) #[N*D]
    var_raw = torch.exp(logvar_raw)
    var_tile = var_raw.unsqueeze(2).repeat(1, 1, self.num_mixture)
    rho_tile = rho.unsqueeze(1).repeat(1, self.y_dim, 1)
    tau_inv = self.tau_inv
    var = (1.0 - torch.pow(rho_tile,2))*var_tile + tau_inv

    # Weight allocation probability pi [N*K]
    pi_logits = self.fc_pi_logits(self.feature) #[N*K]
    pi_temp = F.softmax(pi_logits, dim=1)

    pi_temp = torch.cat((pi_temp[:, 0:1] + self.pi1_bias, pi_temp[:, 1:]), axis=1)

    pi = F.softmax(pi_temp, dim=1)

    return rho, mu, var, pi
    
  def make_sample(self, Q, num_data):
    N = num_data

    #muW = torch.nn.init.normal_(torch.empty(Q, self.y_dim), mean = 0.0, std = 0.1)
    muW = torch.randn(Q, self.y_dim, dtype=torch.float)
    logSigmaW = torch.nn.init.constant_(torch.empty(Q, self.y_dim), -3.0)
    
    muZ = torch.zeros(Q, self.y_dim)
    logSigmaZ = torch.nn.init.constant_(torch.empty(Q, self.y_dim), self.logSigmaZval)
    
    muW_tile = muW.unsqueeze(0).repeat(N,1,1)
    sigmaW_tile = torch.exp(logSigmaW.unsqueeze(0).repeat(N,1,1))
    
    muZ_tile = muZ.unsqueeze(0).repeat(N,1,1)
    sigmaZ_tile = torch.exp(logSigmaZ.unsqueeze(0).repeat(N,1,1))

    return muW_tile, muZ_tile, sigmaW_tile, sigmaZ_tile

  def cholesky(self, num_mixture, Q, rho, num_data, muW_tile, sigmaW_tile, muZ_tile, sigmaZ_tile):
    samplerList = []
    for mix_idx in range(self.num_mixture):
      rho_j = rho[:, mix_idx : mix_idx+1]
      rho_tile = rho_j.unsqueeze(2).repeat(1, Q, self.y_dim)
      
      #epsW = torch.nn.init.normal_(torch.empty(num_data, Q, self.y_dim), mean = 0, std = 1)  #[N,Q,D]
      epsW = torch.randn(num_data, Q, self.y_dim, dtype=torch.float)
      W = muW_tile + torch.sqrt(sigmaW_tile)*epsW
      
      #epsZ = torch.nn.init.normal_(torch.empty(num_data, Q, self.y_dim), mean = 0, std = 1)
      epsZ = torch.randn(num_data, Q, self.y_dim, dtype=torch.float)
      Z = muZ_tile + torch.sqrt(sigmaZ_tile)*epsZ

      #Cholesky
      Y = rho_tile*muW_tile + (1.0 - torch.pow(rho_tile,2))\
                               *(rho_tile*torch.sqrt(sigmaZ_tile)/torch.sqrt(sigmaW_tile)\
                                 *(W - muW_tile) + Z*torch.sqrt(1 - torch.pow(rho_tile,2)))
      
      samplerList.append(Y)
    return torch.stack(samplerList)


    #muW_tile = 
def make_layers(in_channels, h_dim, filter_size, max_pools, activation, batch_norm =False):
    layers = []

    #mnist
    for h_idx in range(len(h_dim)):
      fs = filter_size[h_idx]
      hidden = h_dim[h_idx]
      conv2d = nn.Conv2d(in_channels, hidden, kernel_size = fs, padding = 1)
      if batch_norm:
        layers += [conv2d, nn.BatchNorm2d(hidden), activation]
      else:
        layers += [conv2d, activation]
      in_channels = hidden
     
      max_pool = max_pools[h_idx]
      if max_pool > 1:
        layers += [nn.MaxPool2d(max_pool)]
    return nn.Sequential(*layers)


def ChoiceNet_Mnist():
  return ChoiceNet(make_layers(in_channels, h_dim, filter_size, max_pools, activation, batch_norm ),
                   y_dim, num_mixture, feature_dim, logSigmaZval, tau_inv, pi1_bias)
  

In [0]:



args = {}
kwargs = {}
args['batch_size'] = 1000
args['test_batch_size'] = 1000
args['epochs'] = 10
args['lr'] = 0.01
args['momentum'] = 0.5

args['seed'] = 1
args['log_interval'] = 10
args['cuda'] = False

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                                                 transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,), (0.3081,))
                   ])),
                   batch_size = args['batch_size'], shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, download=True,
                   transform=transforms.Compose([
                                                 transforms.ToTensor(),
                                                 transforms.Normalize((0.1307,), (0.3081,))
                   ])),
                   batch_size = args['test_batch_size'], shuffle=True, **kwargs)


In [0]:



def train(epoch):
  model.train()
  for batch_idx, (data,target) in enumerate(train_loader):
    if args['cuda']:
      data, target = data.cuda(), target.cuda()
    
    temp = torch.LongTensor(1000,1).random_() % 10
    y_onehot = torch.FloatTensor(1000,10)

    y_onehot.zero_()
    y_onehot.scatter_(1, temp, 1)
    
    data, target = Variable(data), Variable(y_onehot)
    
    #GRAD_CLIP = True, USE_SGD = False

    optimizer.zero_grad()

    rho, mu, var, pi = model(data)

    loss = MDNloss(len(data), rho, mu, var, pi, target, y_dim=10, num_mixture=10, logsumexp_coef= 1e-2, kl_reg_coef= 1e-4).forward()
    loss.requires_grad=True
    loss.backward()

    optimizer.step()

    if batch_idx%args['log_interval'] == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. *batch_idx/len(train_loader), loss.data
      ))


In [0]:
def MDNloss(num_data, rho, mu, var, pi, target, y_dim, num_mixture, logsumexp_coef, kl_reg_coef):
  yhat = mu + torch.sqrt(var)*torch.nn.init.normal_(torch.empty(num_data, y_dim, num_mixture))
  target_tile = target.unsqueeze(-1).repeat(1, 1, num_mixture)
  #target_tile = torch.unsqueeze(target,2)
  pi_tile = pi.unsqueeze(1).repeat(1, y_dim, 1)

  yhat_normalized = F.softmax(yhat, dim=1)
  _loss_fit = torch.sum(pi_tile*yhat_normalized*target_tile, axis=[1,2])
  loss_fit = torch.mean(_loss_fit)

  _loss_reg = pi*torch.logsumexp(yhat,axis=[1])
  __loss_reg = torch.sum(_loss_reg,axis=[1])
  loss_reg = logsumexp_coef*torch.mean(__loss_reg)

  _eps = 1e-8
  _kl_reg = kl_reg_coef*torch.sum(rho*(torch.log(pi+_eps) - torch.log(rho+_eps)), axis=1)
  kl_reg = torch.mean(_kl_reg)

  return torch.mean(loss_fit + loss_reg)

In [0]:
import math
class MDNloss(nn.Module):
  def __init__(self, num_data, rho, mu, var, pi, target, y_dim, num_mixture, logsumexp_coef, kl_reg_coef):
    super(MDNloss, self).__init__()
    self.num_data = num_data
    self.rho = Variable(rho)
    self.mu = Variable(mu)
    self.var = Variable(var)
    self.pi = Variable(pi)
    self.target = Variable(target)
    self.y_dim = y_dim
    self.num_mixture = num_mixture
    self.logsumexp_coef = logsumexp_coef
    self.kl_reg_coef = kl_reg_coef
    self.yhat = self.mu + torch.sqrt(self.var)*torch.randn(self.num_data, self.y_dim, self.num_mixture)

  def forward(self):
    target_tile = self.target.unsqueeze(-1).repeat(1, 1, self.num_mixture)
    #target_tile = torch.unsqueeze(target,2)
    pi_tile = self.pi.unsqueeze(1).repeat(1, self.y_dim, 1)

    yhat_normalized = F.softmax(self.yhat, dim=1)
    _loss_fit = torch.sum(pi_tile*yhat_normalized*target_tile, axis=[1,2])
    loss_fit = torch.mean(_loss_fit)

    _loss_reg = self.pi*torch.logsumexp(self.yhat,axis=[1])
    __loss_reg = torch.sum(_loss_reg,axis=[1])
    loss_reg = self.logsumexp_coef*torch.mean(__loss_reg)

    _eps = 1e-8
    _kl_reg = self.kl_reg_coef*torch.sum(self.rho*(torch.log(self.pi+_eps) - torch.log(self.rho+_eps)), axis=1)
    kl_reg = torch.mean(_kl_reg)
    # prob = self.pi*self.g_p(self.var,self.mu,self.target)
    # nll = -torch.log(torch.sum(prob,dim=1))

    return torch.mean(loss_fit + loss_reg)

  def g_p(self, sigma, mu, target):
    ONEOVERSQRT2PI = 1.0/math.sqrt(2*math.pi)
    target = target.unsqueeze(1).expand_as(sigma)
    ret = ONEOVERSQRT2PI * torch.exp(-0.5*((target-mu)/sigma)**2)/sigma
    return torch.prod(ret,2)

  


In [66]:
model = ChoiceNet(make_layers(in_channels=1, h_dim=[64,64], filter_size=[3,3], max_pools=[2,2], activation=torch.nn.ReLU(), batch_norm=True ),y_dim=10, num_mixture=10, feature_dim=128, logSigmaZval=-2, tau_inv=1e-4, pi1_bias=0.0)

optimizer = optim.Adam(model.parameters(), lr=args['lr'])

for epoch in range(1, args['epochs']+1):
  train(epoch)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/60000 (0%)]	Loss: 0.167755
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.160112
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.171089
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.157319
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.171341
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.163990
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.169000
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.179868
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.165136
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.155806
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.186804
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.177507
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.182849
Train Epoch: 3 [10000/60000 (17%)]	Loss: 0.174369
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.174780
Train Epoch: 3 [30000/60000 (50%)]	Loss: 0.173424
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.162308
Train Epoch: 3 [50000/60000 (83%)]	Loss: 0.165289
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.159732
Train Epoch: 4 [10000/60000 (17%)]	Loss: 0.172748
Train Epoch: 4 [2000